In [53]:
# importing all the necessary libaries
import os
import csv
import time
import string
import keyboard
import pandas as pd
from re import search
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [54]:
# define your school name right here:

School_name = "Massachusetts Institute of Technology"

In [55]:
# defining the webdriver and config btw this code will be almost the same in all of your selenium scripts
options = Options()

# !!! blocking browser notifications !!!
prefs = {"profile.default_content_setting_values.notifications": 2}
options.add_experimental_option("prefs", prefs)

# starting in maximized window
options.add_argument("start-maximized")
options.add_argument("--disable-default-apps")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\tahae\.wdm\drivers\chromedriver\win32\101.0.4951.41]


In [56]:
driver.get("https://www.linkedin.com/login/")

In [57]:
username = os.environ.get("ln_us")
password = os.environ.get("ln_ps")

In [ ]:
username_field = driver.find_element(By.ID,"username")
username_field.send_keys(username)

password_field = driver.find_element(By.ID,"password")
password_field.send_keys(password)

password_field.submit()

In [ ]:
'''search_bar = driver.find_element(By.CLASS_NAME,"search-global-typeahead__input")
search_bar.send_keys(School_name)
time.sleep(1)
keyboard.press_and_release('enter')'''

In [ ]:
driver.get("https://www.linkedin.com/search/results/schools/?keywords="+School_name)
time.sleep(3)

In [ ]:
#first_result = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, """//*[@id="main"]/div/div/div[1]/ul/li/div/div/div[2]/div/div[1]/div/span/span/a""")))
#first_result.click()
school_url = driver.find_element(By.XPATH, """//*[@id="main"]/div/div/div[1]/ul/li/div/div/div[2]/div/div[1]/div/span/span/a""").get_attribute('href')
driver.get(school_url)

In [ ]:
driver.get(driver.current_url+"people")
time.sleep(3)

In [ ]:
count = 0
items_urls = []
testing = []
cards_links = driver.find_elements(By.CLASS_NAME, "org-people-profile-card__profile-info")
print(len(cards_links))
for card in cards_links: print(card.text+"\n----------------------------------\n")


In [ ]:
print("lengh is: "+str(len(cards_links)))
for card in cards_links: print(card.text+"\n----------------------------------\n")

In [ ]:
count = 0
items_urls = []
testing = []

cards = driver.find_elements(By.XPATH, "//body[@class='render-mode-BIGPIPE nav-v2 ember-application icons-loaded boot-complete']/div[@class='application-outlet']/div[@class='authentication-outlet']/div[@class='organization-outlet relative']/div[2]/div[1]/div[2]/main[1]/div[2]/div[1]/div[2]/div[1]/div[1]/ul[1]/li/section[1]/div[1]/div[1]/div[1]/a")

for card in cards:
    print(card.get_attribute('href'))


In [ ]:

count = 0
items_urls = []
testing = []
while len(items_urls) < 1000:

    # keyboard.press_and_release('down arrow')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/.1);")
    time.sleep(.7)
    cards_links = driver.find_elements(By.XPATH, "//body[@class='render-mode-BIGPIPE nav-v2 ember-application icons-loaded boot-complete']/div[@class='application-outlet']/div[@class='authentication-outlet']/div[@class='organization-outlet relative']/div[2]/div[1]/div[2]/main[1]/div[2]/div[1]/div[2]/div[1]/div[1]/ul[1]/li/section[1]/div[1]/div[1]/div[1]/a")
    if cards_links == testing:
        break
    for card in cards_links:
        try:
            items_urls.append(card.get_attribute('href'))
        except:
            continue
    testing = driver.find_elements(By.XPATH, "//body[@class='render-mode-BIGPIPE nav-v2 ember-application icons-loaded boot-complete']/div[@class='application-outlet']/div[@class='authentication-outlet']/div[@class='organization-outlet relative']/div[2]/div[1]/div[2]/main[1]/div[2]/div[1]/div[2]/div[1]/div[1]/ul[1]/li/section[1]/div[1]/div[1]/div[1]/a")
    cards_links = []
print(items_urls)

In [ ]:
profilesID = items_urls
names = []
companies = []
locations = []
shit = []

for profileID in profilesID:
    fulllink =  profileID
    driver.get(fulllink)
    time.sleep(1)
    
    name = driver.find_element(By.CLASS_NAME, "text-heading-xlarge").text
    names.append(name)
                                                                                              
    location = driver.find_element(By.XPATH, "//span[@class='text-body-small inline t-black--light break-words']").text
    locations.append(location)
    
    company = driver.find_element(By.XPATH, "//h2[@class='pv-text-details__right-panel-item-text hoverable-link-text break-words text-body-small inline']").text
    companies.append(company)
    

In [ ]:
ids = [i for i in range(len(items_urls))]
df = pd.DataFrame(list(zip(ids, companies, locations)), columns =['id', 'companies', 'locations']) 

In [ ]:
df

In [ ]:
df.to_csv("inpt.csv")

In [ ]:
"""profilesID = items_urls
names = []
companies = []
locs = []

for profileID in profilesID:
    fulllink =  profileID
    driver.get(fulllink)
    time.sleep(8)
    src = driver.page_source
    soup = BeautifulSoup(src, 'lxml')

    #scraping locations
    name_div = soup.find('h1', {'class': 'text-heading-xlarge'}).text
    print(name_div)
    loc = []
    for i in range(len(name_div)):
        span = name_div[i].find_all('span')
        loc.append(span[1].get_text().strip())
    locs.append(loc)

    #scraping companies
    name_div = soup.find_all('p', {'class': 'pv-entity__secondary-title t-14 t-black t-normal'})
    company = []
    for i in range(len(name_div)):
        company.append(name_div[i].get_text().strip())
    companies.append(company)
    print(companies)"""

In [ ]:
"""while len(items_urls) < 100:

    # keyboard.press_and_release('down arrow')
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight/.1);")
    time.sleep(.7)
    cards_links = driver.find_elements(By.CLASS_NAME, "org-people-profile-card__profile-info")
    if cards_links == testing:
        count += 1
    if count == 10:
        break
    for card in cards_links:
        try:
            item_url = card.get_attribute('href')
            items_urls.append(item_url)
        except:
            continue
    testing = driver.find_elements_by_xpath(
        "//section[@class='annonces ng-star-inserted']//a")
    cards_links = []
    print(len(items_urls))"""